In [1]:
n=5 #Prime router number
N=10 #regular router number
m=3 #shortest path number

In [2]:
file_name='full_graph_{}_{}.txt'.format(n,N)
file_name_shortest_path='full_graph_shortest_path_{}_{}.txt'.format(n,N)
file_best_clustering='full_graph_best_clustering_{}_{}.txt'.format(n,N)

# Reading graph

In [3]:
con=[[] for _ in range(n+N+1)]
edges=set()

In [4]:
with open('test/'+file_name, 'r') as f:
    
    n=int(f.readline())
    next(f)
    
    #f.write('\n')
    for line in f:
        #print('line',line)
        x,y,w=map(float,line.split())
        x,y=int(x),int(y)
        #print(x,y,w)
        con[x].append((y,w))
        con[y].append((x,w))
        edges.add((x,y,w))

           
    f.close()

In [5]:
edges

{(1, 11, 0.8550095815477146),
 (1, 13, 0.13651902276258543),
 (1, 14, 0.4552114523664397),
 (1, 15, 0.7193580942721072),
 (2, 3, 0.3358722859296459),
 (2, 4, 0.8599331292728976),
 (2, 5, 0.24832691505262272),
 (2, 6, 0.4303409705286523),
 (2, 7, 0.8600341979033854),
 (2, 8, 0.5710015751835147),
 (2, 9, 0.24209810441616286),
 (2, 10, 0.9842410387580762),
 (2, 11, 0.9559853575911531),
 (2, 12, 0.38665511612196435),
 (2, 13, 0.7675423843816244),
 (3, 2, 0.3358722859296459),
 (3, 4, 0.3671880920795716),
 (3, 7, 0.6877173709908387),
 (3, 9, 0.33828160798887774),
 (3, 13, 0.7211358616091744),
 (3, 14, 0.6729130828832756),
 (3, 15, 0.7713415032141654),
 (4, 2, 0.8599331292728976),
 (4, 3, 0.3671880920795716),
 (4, 5, 0.2688155114523566),
 (4, 6, 0.10372897412869164),
 (4, 7, 0.8126127174604049),
 (4, 8, 0.9899069494890694),
 (4, 9, 0.7434852828685689),
 (4, 10, 0.9561471020946433),
 (4, 11, 0.574548906236377),
 (4, 12, 0.6756606494147235),
 (4, 13, 0.8178947166091707),
 (5, 2, 0.2483269150526

# Reading clustering graph

In [6]:
clust=[0]*(N+1)

In [7]:
with open('test/'+file_best_clustering, 'r') as f:
    
    n,N=map(int,f.readline().split())
    next(f)
    
    #f.write('\n')
    i=1
    for line in f:
        i,j=map(int,line.split())
        
        clust[i]=j
        i+=1
        
        

           
    f.close()

In [8]:
clust[1:]

[12, 15, 13, 13, 12, 15, 11, 14, 11, 15]

# Network simulation

In [9]:
packet_size=0.001

In [11]:
class Packet(object):
    """ A very simple class that represents a packet.
        This packet will run through a queue at a switch output port.
        We use a float to represent the size of the packet in bytes so that
        we can compare to ideal M/M/1 queues.

        Parameters
        ----------
        time : float
            the time the packet arrives at the output queue.
        size : float
            the size of the packet in bytes
        id : int
            an identifier for the packet
        src, dst : int
            identifiers for source and destination
        flow_id : int
            small integer that can be used to identify a flow
    """
    def __init__(self, time, size, id, src="a", dst="z", flow_id=0): #, path):
        self.time = time
        self.size = size
        self.id = id
        self.src = src
        self.dst = dst
        self.flow_id = flow_id
        #self.path=path

    def __repr__(self):
        return "id: {}, src: {}, time: {}, size: {}".\
            format(self.id, self.src, self.time, self.size)




In [12]:
class PacketGenerator(object):
    """ Generates packets with given inter-arrival time distribution.
        Set the "out" member variable to the entity to receive the packet.

        Parameters
        ----------
        env : simpy.Environment
            the simulation environment
        adist : function
            a no parameter function that returns the successive inter-arrival times of the packets
        sdist : function
            a no parameter function that returns the successive sizes of the packets
        initial_delay : number
            Starts generation after an initial delay. Default = 0
        finish : number
            Stops generation at the finish time. Default is infinite


    """
    def __init__(self, env, id,  adist, sdist, initial_delay=0, finish=float("inf"), flow_id=0):
        self.id = id
        self.env = env
        self.adist = adist
        self.sdist = sdist
        self.initial_delay = initial_delay
        self.finish = finish
        self.out = None
        self.packets_sent = 0
        self.action = env.process(self.run())  # starts the run() method as a SimPy process
        self.flow_id = flow_id

    def run(self):
        """The generator function used in simulations.
        """
        yield self.env.timeout(self.initial_delay)
        while self.env.now < self.finish:
            # wait for next transmission
            yield self.env.timeout(self.adist())
            self.packets_sent += 1
            p = Packet(self.env.now, self.sdist(), self.packets_sent, src=self.id, flow_id=self.flow_id)
            self.out.put(p)


In [13]:
class PacketSink(object):
    """ Receives packets and collects delay information into the
        waits list. You can then use this list to look at delay statistics.

        Parameters
        ----------
        env : simpy.Environment
            the simulation environment
        debug : boolean
            if true then the contents of each packet will be printed as it is received.
        rec_arrivals : boolean
            if true then arrivals will be recorded
        absolute_arrivals : boolean
            if true absolute arrival times will be recorded, otherwise the time between consecutive arrivals
            is recorded.
        rec_waits : boolean
            if true waiting time experienced by each packet is recorded
        selector: a function that takes a packet and returns a boolean
            used for selective statistics. Default none.

    """
    def __init__(self, env, rec_arrivals=False, absolute_arrivals=False, rec_waits=True, debug=False, selector=None):
        self.store = simpy.Store(env)
        self.env = env
        self.rec_waits = rec_waits
        self.rec_arrivals = rec_arrivals
        self.absolute_arrivals = absolute_arrivals
        self.waits = []
        self.arrivals = []
        self.debug = debug
        self.packets_rec = 0
        self.bytes_rec = 0
        self.selector = selector
        self.last_arrival = 0.0

    def put(self, pkt):
        #print('packet_id',pkt.id)
        
        if not self.selector or self.selector(pkt):
            now = self.env.now
            if self.rec_waits:
                self.waits.append(self.env.now - pkt.time)
            if self.rec_arrivals:
                if self.absolute_arrivals:
                    self.arrivals.append(now)
                else:
                    self.arrivals.append(now - self.last_arrival)
                self.last_arrival = now
            self.packets_rec += 1
            self.bytes_rec += pkt.size
            if self.debug:
                print(pkt)

In [14]:
from random import expovariate
import simpy

def constArrival():  # Constant arrival distribution for generator 1
    return 1.5

def constArrival2():
    return 2.0

def distSize():
    return expovariate(0.01)

In [15]:
env = simpy.Environment()  # Create the SimPy environment
# Create the packet generators and sink
ps = PacketSink(env, debug=True)  # debugging enable for simple output
pg = PacketGenerator(env, "EE283", constArrival, distSize)
pg2 = PacketGenerator(env, "SJSU", constArrival2, distSize)
# Wire packet generators and sink together
pg.out = ps
pg2.out = ps
env.run(until=20) # Run it

id: 1, src: EE283, time: 1.5, size: 57.66229390366405
id: 1, src: SJSU, time: 2.0, size: 133.3653782299575
id: 2, src: EE283, time: 3.0, size: 218.69142508294908
id: 2, src: SJSU, time: 4.0, size: 125.03733135870962
id: 3, src: EE283, time: 4.5, size: 38.84047930614041
id: 3, src: SJSU, time: 6.0, size: 301.2630092193702
id: 4, src: EE283, time: 6.0, size: 34.80771270940797
id: 5, src: EE283, time: 7.5, size: 206.74691543912112
id: 4, src: SJSU, time: 8.0, size: 3.491919007421487
id: 6, src: EE283, time: 9.0, size: 405.8544017967127
id: 5, src: SJSU, time: 10.0, size: 224.0848566168534
id: 7, src: EE283, time: 10.5, size: 60.38393499004705
id: 6, src: SJSU, time: 12.0, size: 16.03309238266907
id: 8, src: EE283, time: 12.0, size: 198.46986266483697
id: 9, src: EE283, time: 13.5, size: 11.007238362026579
id: 7, src: SJSU, time: 14.0, size: 103.79502686750723
id: 10, src: EE283, time: 15.0, size: 95.35568242946691
id: 8, src: SJSU, time: 16.0, size: 6.56085611484674
id: 11, src: EE283, ti